In [1]:
import tqdm.notebook as tqdm
import numpy as np
import scipy
import sklearn

# Spark

In [2]:
! /home/jovyan/start-hadoop.sh

jovyan
 * Starting OpenBSD Secure Shell server sshd
start-stop-daemon: unable to set gid to 0 (Operation not permitted)
   ...fail!
 * sshd is running
Starting namenodes on [localhost]
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: namenode is running as process 156.  Stop it first and ensure /tmp/hadoop-jovyan-namenode.pid file is empty before retry.
Starting datanodes
localhost: Warning: Permanently added 'localhost' (ED25519) to the list of known hosts.
localhost: datanode is running as process 274.  Stop it first and ensure /tmp/hadoop-jovyan-datanode.pid file is empty before retry.
Starting secondary namenodes [2cb88d0197b3]
2cb88d0197b3: Warning: Permanently added '2cb88d0197b3' (ED25519) to the list of known hosts.
2cb88d0197b3: secondarynamenode is running as process 462.  Stop it first and ensure /tmp/hadoop-jovyan-secondarynamenode.pid file is empty before retry.
Starting resourcemanager
resourcemanager is running as process

In [3]:
# connect, context, session

import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-05-07 13:41:26,884 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


## HDFS

In [4]:
! hdfs dfs -df -h

Filesystem                Size     Used  Available  Use%
hdfs://localhost:9000  196.8 G  291.6 M    153.2 G    0%


In [5]:
! hdfs dfs -ls /

Found 2 items
drwxrwx---   - root   supergroup          0 2023-05-07 13:41 /tmp
drwxr-xr-x   - jovyan supergroup          0 2023-05-07 13:41 /user


## Outbrain click prediction dataseet

https://www.kaggle.com/c/outbrain-click-prediction/data - you need to register here

In [ ]:
# https://github.com/Kaggle/kaggle-api
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
# instruction where to get ~/.kaggle/kaggle.json file

  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [6]:
! mkdir -p ~/.kaggle

In [17]:
%%writefile ~/.kaggle/kaggle.json
{"username":"juliakravchenko","key":"c2b4fad3ce13b57b2e7f1a028e6f9410"}

Overwriting /home/jovyan/.kaggle/kaggle.json


In [18]:
! chmod 600 ~/.kaggle/kaggle.json

In [19]:
! pip install -U urllib3 kaggle==1.5.3
! kaggle competitions download -c outbrain-click-prediction -f page_views_sample.csv.zip
! kaggle competitions download -c outbrain-click-prediction -f documents_topics.csv.zip 

  Using cached urllib3-2.0.2-py3-none-any.whl (123 kB)
100%|████████████████████████████████████████| 149M/149M [00:06<00:00, 30.7MB/s]
100%|████████████████████████████████████████| 149M/149M [00:06<00:00, 24.8MB/s]
 99%|███████████████████████████████████████▋| 120M/121M [00:04<00:00, 31.0MB/s]
100%|████████████████████████████████████████| 121M/121M [00:04<00:00, 26.5MB/s]


In [20]:
!unzip '*.zip'
!rm -rf *.zip

Archive:  documents_topics.csv.zip
  inflating: documents_topics.csv    

Archive:  page_views_sample.csv.zip
  inflating: page_views_sample.csv   

2 archives were successfully processed.


In [22]:
! hdfs dfs -put page_views_sample.csv
! hdfs dfs -put documents_topics.csv

put: `page_views_sample.csv': File exists
put: `documents_topics.csv': File exists


In [24]:
page_views_sample = se.read.csv("page_views_sample.csv", header=True)
page_views_sample.show(5)

+--------------+-----------+---------+--------+------------+--------------+
|          uuid|document_id|timestamp|platform|geo_location|traffic_source|
+--------------+-----------+---------+--------+------------+--------------+
|1fd5f051fba643|        120| 31905835|       1|          RS|             2|
|8557aa9004be3b|        120| 32053104|       1|       VN>44|             2|
|c351b277a358f0|        120| 54013023|       1|       KR>12|             1|
|8205775c5387f9|        120| 44196592|       1|       IN>16|             2|
|9cb0ccd8458371|        120| 65817371|       1|   US>CA>807|             2|
+--------------+-----------+---------+--------+------------+--------------+
only showing top 5 rows



In [25]:
documents_topics = se.read.csv("documents_topics.csv", header=True)
documents_topics.show(5)

+-----------+--------+------------------+
|document_id|topic_id|  confidence_level|
+-----------+--------+------------------+
|    1595802|     140|0.0731131601068925|
|    1595802|      16|0.0594164867373976|
|    1595802|     143|0.0454207537554526|
|    1595802|     170|0.0388674285182961|
|    1524246|     113| 0.196450402209685|
+-----------+--------+------------------+
only showing top 5 rows



## Evaluation Assignment

Data: outbrain click prediction

Tasks:
Using Spark RDD, DataFrame API and Python, calculate:

**1**. Top 10 most visited document_ids in the page_views_sample log

**2**. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

**3***. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

The submission format is the result.json json file with top_10_documents, users and top_10_topics keys.
For TOP-10 results, the answer must be written in the form of a sheet ordered from TOP-1 to TOP-10 with an id.

result.json example:

    {
        "top_10_documents": [
            111,
            222,
            333,
            ...,
            1010
        ],
        "users": 10000,
        "top_10_topics": [
            11,
            22,
            33,
            ...,
            101
        ]
    }

In [26]:
# 1. Top 10 most visited document_ids in the page_views_sample log

from pyspark.sql.functions import col

top_10_documents = page_views_sample.groupBy("document_id") \
                  .count() \
                  .sort(col("count").desc()) \
                  .limit(10)
top_10_documents_list = [int(row['document_id']) for row in top_10_documents.collect()]

In [27]:
top_10_documents.show()

+-----------+------+
|document_id| count|
+-----------+------+
|    1811567|429551|
|        234|179692|
|      42744|156231|
|    1858440|112140|
|    1780813|109624|
|      60164|104941|
|    1790442| 91420|
|    1877626| 80309|
|    1821895| 79956|
|     732651| 74630|
+-----------+------+



In [28]:
# 2. How many users have at least 2 different traffic_sources in the page_views_sample log (note the value is not a count, it's an encoded enum)

from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, countDistinct


users = (page_views_sample.groupBy("uuid")
         .agg(countDistinct("traffic_source").alias("distinct_traffic_sources"))
         .where("distinct_traffic_sources >= 2")
         .count())
users

98080

In [29]:
# 3*. Top 10 most visited topic_ids in page_views_sample log (use documents_topics table)

top_10_topics = (page_views_sample.join(documents_topics, on="document_id", how="inner")
                 .groupBy("topic_id")
                 .count()
                 .orderBy("count", ascending=False)
                 .limit(10)
                 .rdd.map(lambda row: int(row.topic_id))
                 .collect())
top_10_topics

[20, 16, 216, 136, 140, 143, 36, 97, 8, 269]

In [33]:
import json

result = {"top_10_documents": top_10_documents_list,
          "users": users,
          "top_10_topics": top_10_topics}

with open("result.json", "w") as f:
    json.dump(result, f)

In [34]:
result

{'top_10_documents': [1811567,
  234,
  42744,
  1858440,
  1780813,
  60164,
  1790442,
  1877626,
  1821895,
  732651],
 'users': 98080,
 'top_10_topics': [20, 16, 216, 136, 140, 143, 36, 97, 8, 269]}

In [35]:
!curl -F file=@result.json "51.250.54.133:80/MDS-LSML1/JuliaKravch/w4/1"

1.0
Well done!
